In [1]:
import pandas as pd
from indra.literature import pubmed_client
from orcid_downloader.api import ground_researcher
from orcid_downloader.pubmed import get_metadata_batched, process_pmid_results, search_pubmed_for_name

In [ ]:
%%time
ground_researcher("Steven B Bradfute")

In [ ]:
ground_researcher("Steven Bradfute")

In [3]:
BRADFUTE_ORCID = "0000-0002-1985-751X"
bradfute_pmids = search_pubmed_for_name("Steven B Bradfute")
bradfute_records = get_metadata_batched(bradfute_pmids)

(
    bradfute_annotations,
    bradfute_orcid_to_paper,
    bradfute_ambiguous,
    bradfute_misses,
) = process_pmid_results(bradfute_records)

print(
    f"There are {len(bradfute_annotations):,} paper-author annotations "
    f"with {len(bradfute_orcid_to_paper)} unique authors and {len(bradfute_pmids):,} "
    f"papers."
)

n_bradfute_annotated = len(bradfute_orcid_to_paper[BRADFUTE_ORCID])
print(
    f"There are {n_bradfute_annotated:,} ({n_bradfute_annotated/len(bradfute_pmids):.1%}) "
    f"papers annotated with Bradfute."
)

In [ ]:
from IPython.display import Markdown

Markdown(pd.DataFrame(bradfute_misses.most_common()).to_markdown())

In [ ]:
from IPython.display import Markdown

Markdown(pd.DataFrame(bradfute_ambiguous.most_common()).to_markdown())